<a href="https://colab.research.google.com/github/leehyunggeunkeun/pytorch-study/blob/master/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MNIST 데이터셋으로 softmax classfication

In [0]:
#edwith 부스트코스 파이토치 강의를 보면서 필사하고 느낀점 및 방법들을 까먹지 않게 정리함...
#출처 edwith 부스트 코스 pytorch 강의 

In [0]:
#필요한 라이브러리 불러오기

#기본
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#기존 데이터셋 불러오기
import torchvision.datasets as dsets         
import torchvision.transforms as transforms  #다양한 이미지 변환

import random


conv.torch.nn.Conv2d()

In [0]:
#gpu 사용하기

device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
#hyperparameters

lr = 0.001
training_epochs = 15
batch_size = 100

In [0]:
mnist_train = dsets.MNIST(root='MNIST_data/',train=True,transform=transforms.ToTensor(),download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',train=False,transform=transforms.ToTensor(),download=True)

In [0]:
#데이터셋 불러오기

dataloader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [0]:
#layer를 한번 쌓아보자....

linear1=torch.nn.Linear(784,256,bias=True)
linear2=torch.nn.Linear(256,128,bias=True)
linear3=torch.nn.Linear(128,64,bias=True)
linear4=torch.nn.Linear(64,32,bias=True)    
linear5=torch.nn.Linear(32,10,bias=True) 

relu=torch.nn.ReLU()



In [8]:
#초기화
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)
torch.nn.init.normal_(linear4.weight)
torch.nn.init.normal_(linear5.weight) 

Parameter containing:
tensor([[-0.8760, -0.2428, -0.0590,  0.1154,  0.1392, -2.1023, -1.3665,  1.6940,
         -0.3535, -0.4622, -0.9144, -2.4040, -1.0246,  1.2613, -0.8845, -0.5964,
          0.8419, -0.2984, -0.2444, -0.7330,  0.6643,  0.3606,  0.0785, -0.0251,
          0.1463,  0.6727, -0.9334,  0.0913, -0.2854,  1.2619,  0.0364,  1.2454],
        [-0.4429,  0.1839,  1.3681, -0.0368, -0.1383,  1.3645,  0.1795,  0.3320,
         -0.7304,  1.1410,  1.8497,  1.3517, -3.2752,  2.0577,  0.7026,  0.6041,
          0.2571,  1.9346, -0.5456, -1.4699,  0.6691, -3.0588, -1.0069, -0.1531,
         -2.0676,  1.0222,  0.0523, -0.5590,  0.2663,  0.7497, -0.9099,  1.4027],
        [ 0.3852,  2.2101, -1.5015, -0.6829,  0.3430, -0.5899, -0.5412, -1.3859,
          1.5074, -0.9125, -0.4844, -0.5480, -0.4807, -0.5619,  0.5002,  1.0479,
          1.5341, -0.9516,  0.0034,  0.6849,  0.0748, -0.2988, -0.8951,  0.0170,
          0.1203, -0.2802, -1.2814,  1.2865,  0.2399,  0.3876,  1.4302, -0.7666],
   

In [0]:
model = torch.nn.Sequential(linear1, relu,linear2, relu,linear3,relu,linear4,relu,linear5).to(device)

In [0]:
criterion = torch.nn.CrossEntropyLoss().to(device)     
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [11]:
total_batch=len(dataloader)


for epoch in range(training_epochs):
    avg_cost=0

    for x,y in dataloader:
        x=x.view(-1,784).to(device)
        y=y.to(device)

        optimizer.zero_grad()
        hypothesis=model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')


Epoch: 0001 cost = 1525.619140625
Epoch: 0002 cost = 276.732330322
Epoch: 0003 cost = 155.909927368
Epoch: 0004 cost = 101.592491150
Epoch: 0005 cost = 71.055374146
Epoch: 0006 cost = 51.889701843
Epoch: 0007 cost = 38.824405670
Epoch: 0008 cost = 29.453872681
Epoch: 0009 cost = 22.782083511
Epoch: 0010 cost = 17.747697830
Epoch: 0011 cost = 13.830949783
Epoch: 0012 cost = 11.233580589
Epoch: 0013 cost = 9.198373795
Epoch: 0014 cost = 7.390275002
Epoch: 0015 cost = 6.056800365
Learning finished


In [12]:
# Test the model using test sets
with torch.no_grad():
    
    x_test = mnist_test.data.view(-1, 784).float().to(device)
    y_test = mnist_test.targets.to(device)

    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    x_single_data = mnist_test.data[r:r + 1].view(-1, 784).float().to(device)
    y_single_data = mnist_test.targets[r:r + 1].to(device)

    print('Label: ', y_single_data.item())
    single_prediction = model(x_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


Accuracy: 0.9131999611854553
Label:  9
Prediction:  9


In [13]:
from sklearn.datasets import load_iris

df = load_iris()
df

{'DESCR': '.. _iris_dataset:\n\nIris plants dataset\n--------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n                \n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)\n    ===========

cs231n 강의에서도 딴거 말고 걍 Batch Normalization 하라 했음...
Batch Normalization 에 대해 알아보자

In [0]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [0]:
# parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 32

In [0]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [0]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [0]:
# nn layers
linear1 = torch.nn.Linear(784, 32, bias=True)
linear2 = torch.nn.Linear(32, 32, bias=True)
linear3 = torch.nn.Linear(32, 10, bias=True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(32)
bn2 = torch.nn.BatchNorm1d(32)

nn_linear1 = torch.nn.Linear(784, 32, bias=True)
nn_linear2 = torch.nn.Linear(32, 32, bias=True)
nn_linear3 = torch.nn.Linear(32, 10, bias=True)

In [0]:
# model
bn_model = torch.nn.Sequential(linear1, bn1, relu,
                            linear2, bn2, relu,
                            linear3).to(device)
nn_model = torch.nn.Sequential(nn_linear1, relu,
                               nn_linear2, relu,
                               nn_linear3).to(device)

In [0]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)
nn_optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

In [22]:

# Save Losses and Accuracies every epoch
# We are going to plot them later
train_losses = []
train_accs = []

valid_losses = []
valid_accs = []

train_total_batch = len(train_loader)
test_total_batch = len(test_loader)
for epoch in range(training_epochs):
    bn_model.train()  # set the model to train mode

    for X, Y in train_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        bn_optimizer.zero_grad()
        bn_prediction = bn_model(X)
        bn_loss = criterion(bn_prediction, Y)
        bn_loss.backward()
        bn_optimizer.step()

        nn_optimizer.zero_grad()
        nn_prediction = nn_model(X)
        nn_loss = criterion(nn_prediction, Y)
        nn_loss.backward()
        nn_optimizer.step()

    with torch.no_grad():
        bn_model.eval()     # set the model to evaluation mode

        # Test the model using train sets
        bn_loss, nn_loss, bn_acc, nn_acc = 0, 0, 0, 0
        for i, (X, Y) in enumerate(train_loader):
            X = X.view(-1, 28 * 28).to(device)
            Y = Y.to(device)

            bn_prediction = bn_model(X)
            bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y
            bn_loss += criterion(bn_prediction, Y)
            bn_acc += bn_correct_prediction.float().mean()

            nn_prediction = nn_model(X)
            nn_correct_prediction = torch.argmax(nn_prediction, 1) == Y
            nn_loss += criterion(nn_prediction, Y)
            nn_acc += nn_correct_prediction.float().mean()

        bn_loss, nn_loss, bn_acc, nn_acc = bn_loss / train_total_batch, nn_loss / train_total_batch, bn_acc / train_total_batch, nn_acc / train_total_batch

        # Save train losses/acc
        train_losses.append([bn_loss, nn_loss])
        train_accs.append([bn_acc, nn_acc])
        print(
            '[Epoch %d-TRAIN] Batchnorm Loss(Acc): bn_loss:%.5f(bn_acc:%.2f) vs No Batchnorm Loss(Acc): nn_loss:%.5f(nn_acc:%.2f)' % (
            (epoch + 1), bn_loss.item(), bn_acc.item(), nn_loss.item(), nn_acc.item()))
        # Test the model using test sets
        bn_loss, nn_loss, bn_acc, nn_acc = 0, 0, 0, 0
        for i, (X, Y) in enumerate(test_loader):
            X = X.view(-1, 28 * 28).to(device)
            Y = Y.to(device)

            bn_prediction = bn_model(X)
            bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y
            bn_loss += criterion(bn_prediction, Y)
            bn_acc += bn_correct_prediction.float().mean()

            nn_prediction = nn_model(X)
            nn_correct_prediction = torch.argmax(nn_prediction, 1) == Y
            nn_loss += criterion(nn_prediction, Y)
            nn_acc += nn_correct_prediction.float().mean()

        bn_loss, nn_loss, bn_acc, nn_acc = bn_loss / test_total_batch, nn_loss / test_total_batch, bn_acc / test_total_batch, nn_acc / test_total_batch

        # Save valid losses/acc
        valid_losses.append([bn_loss, nn_loss])
        valid_accs.append([bn_acc, nn_acc])
        print(
            '[Epoch %d-VALID] Batchnorm Loss(Acc): bn_loss:%.5f(bn_acc:%.2f) vs No Batchnorm Loss(Acc): nn_loss:%.5f(nn_acc:%.2f)' % (
                (epoch + 1), bn_loss.item(), bn_acc.item(), nn_loss.item(), nn_acc.item()))
        print()

print('Learning finished')

[Epoch 1-TRAIN] Batchnorm Loss(Acc): bn_loss:0.13505(bn_acc:0.96) vs No Batchnorm Loss(Acc): nn_loss:0.17104(nn_acc:0.95)
[Epoch 1-VALID] Batchnorm Loss(Acc): bn_loss:0.14253(bn_acc:0.95) vs No Batchnorm Loss(Acc): nn_loss:0.19548(nn_acc:0.94)

[Epoch 2-TRAIN] Batchnorm Loss(Acc): bn_loss:0.10647(bn_acc:0.97) vs No Batchnorm Loss(Acc): nn_loss:0.19554(nn_acc:0.94)
[Epoch 2-VALID] Batchnorm Loss(Acc): bn_loss:0.12260(bn_acc:0.96) vs No Batchnorm Loss(Acc): nn_loss:0.22271(nn_acc:0.94)

[Epoch 3-TRAIN] Batchnorm Loss(Acc): bn_loss:0.09675(bn_acc:0.97) vs No Batchnorm Loss(Acc): nn_loss:0.17482(nn_acc:0.95)
[Epoch 3-VALID] Batchnorm Loss(Acc): bn_loss:0.11440(bn_acc:0.96) vs No Batchnorm Loss(Acc): nn_loss:0.19833(nn_acc:0.94)

[Epoch 4-TRAIN] Batchnorm Loss(Acc): bn_loss:0.08105(bn_acc:0.98) vs No Batchnorm Loss(Acc): nn_loss:0.17992(nn_acc:0.95)
[Epoch 4-VALID] Batchnorm Loss(Acc): bn_loss:0.10669(bn_acc:0.97) vs No Batchnorm Loss(Acc): nn_loss:0.21152(nn_acc:0.94)

[Epoch 5-TRAIN] Batc